In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#필요한 모듈 설치하기
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install speechrecognition

     |████████████████████████████████| 46.9 MB 92.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595736 sha256=39d4d7732382a992cd9bc0c5d002adc2785b006071506c4c4305d47c1c9ff13a
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 83.1 MB/s 
     |████████████████████████████████| 3.0 MB 66.8 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7dvqbvu8
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/p

In [ ]:
# 기본 라이브러리 불러오기
import pandas as pd
import numpy as np
import os
import sys

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

# 경고메세지 숨기기
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np

#kobert라이브러리에서 많이 쓰이는 함수 불러오기
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
#transformers에서 하이퍼파라미터 세팅
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pickle

import speech_recognition as sr

In [ ]:
#GPU 사용(권장)
device = torch.device("cuda:0")
#CPU 사용
#device = torch.device("cpu")

음성 및 텍스트 데이터 준비

In [ ]:
#감성대화정보 엑셀로 불러오기
data_pre = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/감성대화/감성대화말뭉치_table.xlsx')

# 감정데이터중에 필요없는 카테고리('당황') 제거해주기

data_sub = data_pre.loc[(data_pre['감정_대분류'] == "기쁨") | (data_pre['감정_소분류'] == "안달하는") | (data_pre['감정_소분류'] == "성가신") | (data_pre['감정_소분류'] == "툴툴대는") | (data_pre['감정_소분류'] == "방어적인") | (data_pre['감정_소분류'] == "혐오스러운")
 | (data_pre['감정_소분류'] == "비통한") | (data_pre['감정_소분류'] == "슬픔") | (data_pre['감정_소분류'] == "우울한") | (data_pre['감정_소분류'] == "실망한") | (data_pre['감정_소분류'] == "후회되는") | (data_pre['감정_소분류'] == "눈물이 나는")
 | (data_pre['감정_소분류'] == "걱정스러운") | (data_pre['감정_소분류'] == "조심스러운") | (data_pre['감정_소분류'] == "초조한") | (data_pre['감정_소분류'] == "두려운") | (data_pre['감정_소분류'] == "스트레스 받는")
 | (data_pre['감정_소분류'] == "고립된") | (data_pre['감정_소분류'] == "염세적인") | (data_pre['감정_소분류'] == "상처") 
 ]
# 음성이 있고 및 라벨에 맞는 데이터 개수 확인하기
len(data_sub)

2428

In [ ]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
data_sub = data_sub.copy()
data_sub.loc[(data_sub['감정_소분류'] == "안달하는") | (data_sub['감정_소분류'] == "성가신") | (data_sub['감정_소분류'] == "툴툴대는") | (data_sub['감정_소분류'] == "방어적인") | (data_sub['감정_소분류'] == "혐오스러운"), 'Emotion'] = '분노'  #분노
data_sub.loc[(data_sub['감정_소분류'] == "비통한") | (data_sub['감정_소분류'] == "슬픔") | (data_sub['감정_소분류'] == "우울한") | (data_sub['감정_소분류'] == "실망한") | (data_sub['감정_소분류'] == "후회되는") | (data_sub['감정_소분류'] == "눈물이 나는"), 'Emotion'] = '우울'  #우울 
data_sub.loc[(data_sub['감정_소분류'] == "걱정스러운") | (data_sub['감정_소분류'] == "조심스러운") | (data_sub['감정_소분류'] == "초조한") | (data_sub['감정_소분류'] == "두려운") | (data_sub['감정_소분류'] == "스트레스 받는"), 'Emotion'] = '불안'  #불안 
data_sub.loc[(data_sub['감정_소분류'] == "고립된") | (data_sub['감정_소분류'] == "염세적인") | (data_sub['감정_소분류'] == "상처")  , 'Emotion'] = '자살'  #자살 
data_sub.loc[(data_sub['감정_대분류'] == "기쁨"), 'Emotion'] = '기쁨' #기쁨


# 음성데이터 라벨 및 경로데이터만 뽑아오기
data_path = data_sub.loc[:,['Emotion','NO.', '사람문장1']]
data_path.columns = ['Emotions','Path', 'Text']

# 음성데이터 경로수정해주기(압축푼 폴더로 경로수정)
data_path['Path'] = '/content/drive/MyDrive/Colab Notebooks/감성대화/원천데이터/감성대화말뭉치AI데이터_Wave_남자성우_5000/'+data_path['Path']+'.wav'

data_path.head(5)


,Emotions,Path,Text
84,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...,오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.
85,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...,오늘 시험에 친구가 알려준 문제가 나왔어. 결과가 좋아서 기쁘다.
86,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...,오늘 대학 합격 통지를 받았어. 엄마가 너무 기뻐하는데 나는 눈물이 났어.
87,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...,오늘 졸업식인데 선생님께서 아프셔서 오지 못하셨어. 그동안 감사하다는 말도 못했는데.
88,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...,나 올림피아드에서 금상 받았어! 그동안 고생이 헛되지 않아서 너무 기뻐.


In [ ]:
# 음성데이터를 waveplot, spectrogram 이미지로 표현

def create_waveplot(data, sr, e):
    plt.figure(figsize=(10, 3))
    plt.title('Waveplot for audio with {} emotion'.format(e), size=15)
    librosa.display.waveplot(data, sr=sr)
    plt.show()

def create_spectrogram(data, sr, e):
    # stft 함수는 음성을 시간기반에서 주파수기반으로 변환함
    X = librosa.stft(data)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(12, 3))
    plt.title('Spectrogram for audio with {} emotion'.format(e), size=15)
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')   
    plt.colorbar()

In [ ]:
# 음성데이터 argumentation을 통해서 데이터 수 늘리기 및 오버피팅 방지
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

# 예시
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)

# 음성의 특성추출하는 함수 ( MFCC, MEL, RMSV)

def extract_features(data):

    result = np.array([])

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
#    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
#    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

# 음성의 특성을 추출한 데이터를 축적하는 함수 (Argumentation된 데이터도 같이)
def get_features(path):
    # duration과 offset은 각 오디오 파일의 시작과 끝에서 오디오가 없는 것을 처리
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
    
    # 원래데이터
    res1 = extract_features(data)
    result = np.array(res1)
    # 노이즈가 추가된 데이터
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2)) # 병렬적으로 추가

    # 피칭및 스트레칭된 데이터
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch)
    result = np.vstack((result, res3)) # 병렬적으로 추가

    return result

In [ ]:
# X, Y, Z = [], [], []
# for path, emotion, text in zip(data_path.Path, data_path.Emotions, data_path.Text):
#     feature = get_features(path)
#     for ele in feature:
#         X.append(ele)
#         # 2개의 augmentation이 존재하므로 총 한개의 음성데이터가 3개의 음성데이터가 됨
#         Y.append(emotion)
#         Z.append(text)

In [ ]:
Features = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ai/features3.csv')
# Features = pd.DataFrame(X)
# Features['labels'] = Y
# Features['text'] = Z
# Features.to_csv('features2.csv', index=False)

In [ ]:
X = Features.iloc[: ,:-2].values  # Audio
Y = Features['labels'].values
Z = Features['text'].values  # Text 

array(['기쁨', '기쁨', '기쁨', ..., '기쁨', '기쁨', '기쁨'], dtype=object)

In [ ]:
le = LabelEncoder()
le = le.fit(Y)
Y = le.transform(Y)
# with open('/content/drive/MyDrive/Colab Notebooks/ai/label_encoder.pkl','wb') as f:
#   pickle.dump(le, f)

In [ ]:
# 훈련데이터, 테스트데이터 분리 및 개수 확인
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)
z_train, z_test, _, _ = train_test_split(Z, Y, random_state=0)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, z_train.shape, z_test.shape

((5463, 148), (5463,), (1821, 148), (1821,), (5463,), (1821,))

In [ ]:
# 음성 데이터 스케일 조정
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape
# with open('/content/drive/MyDrive/Colab Notebooks/ai/scaler.pkl','wb') as f:
#   pickle.dump(scaler, f)

((5463, 148), (5463,), (1821, 148), (1821,))

In [ ]:
# 음성 데이터의 차원 모델에 맞게 통일 
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5463, 148, 1), (5463,), (1821, 148, 1), (1821,))

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [ ]:
audio_train_ds = TensorDataset(x_train, y_train)
audio_test_ds = TensorDataset(x_test, y_test)
audio_train_dataloader = torch.utils.data.DataLoader(audio_train_ds, batch_size=64)
audio_test_dataloader = torch.utils.data.DataLoader(audio_test_ds, batch_size=64)

In [ ]:
# 텍스트 데이터 BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, data, label, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in data]
        self.labels = [np.int32(i) for i in label]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# # BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
using cached model


In [ ]:
# with open('/content/drive/MyDrive/Colab Notebooks/ai/kobert_data.pkl', 'wb') as f:
#   bertmodel, vocab = get_pytorch_kobert_model()
#   tokenizer = get_tokenizer()
#   pickle.dump([bertmodel, vocab, tokenizer],f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ai/kobert_data.pkl', 'rb') as f:
  bertmodel, vocab, _= pickle.load(f)

In [ ]:
# #문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

[██████████████████████████████████████████████████]


In [ ]:
# 데이터셋 BERT에 맞게 적용
max_len = 50
text_train_ds = BERTDataset(z_train, y_train, tok, max_len, True, False)
text_test_ds = BERTDataset(z_test, y_test, tok, max_len, True, False)
#bert모델에 넣기위해 torch형식으로 변환
text_train_dataloader = torch.utils.data.DataLoader(text_train_ds, batch_size=64, num_workers=5)
text_test_dataloader = torch.utils.data.DataLoader(text_test_ds, batch_size=64, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


모델 설계 https://discuss.pytorch.org/t/combining-trained-models-in-pytorch/28383/2

In [ ]:
# 파라미터 세팅 ( 문장길이, 모델배치수,에폭수 ...)
max_len = 50
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
learning_rate =  3e-5

1. 음성모델

In [ ]:
# 음성 모델 만들기

class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=x_train.shape[1], out_channels=256, kernel_size=1, stride=1,padding='same')
        self.max_pool1 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding='same')
        self.max_pool2 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv3 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=5, stride=1, padding='same')
        self.max_pool3 = nn.MaxPool1d(5, stride=2, padding = 2)
        self.dropout1 = nn.Dropout(0.2)
        
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding='same')
        self.max_pool4 = nn.MaxPool1d(5, stride=2, padding = 2)
        
        self.fc1 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)

        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)

        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        x = self.dropout1(x)

        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)

        x = torch.flatten(x,1)

        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        
        return x

In [ ]:
model1 = AudioClassifier().to(device)

2. 텍스트모델

In [ ]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=32,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model2 = BERTClassifier(bertmodel,  dr_rate=0.6).to(device)

멀티모달 모델

In [ ]:
# 멀티모달 모델 만들기

class MultimodalClassifier(nn.Module):
    def __init__(self, audioModel, textModel):
        super(MultimodalClassifier, self).__init__()
        self.audioModel = audioModel
        self.textModel = textModel

        self.fc1 = nn.Linear(64,32)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(32, 5)

    def forward(self, x1, token_ids, valid_length, segment_ids):
        x1 = self.audioModel(x1)
        x2 = self.textModel(token_ids, valid_length, segment_ids)
        x = torch.cat((x1,x2), 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        
        x = F.relu(self.fc2(x))
        return x

In [ ]:
# model3 = MultimodalClassifier(model1, model2).to(device)
model3 = torch.load('/content/drive/MyDrive/Colab Notebooks/ai/multimodal_emotion_classification.pt')

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model3.parameters(), lr=0.0001)

In [ ]:
for epoch in range(num_epochs):
  cost, total, correct = 0,0,0
  for (i, (X, Y)), (_, (token_ids, valid_length, segment_ids, _)) in zip(enumerate(audio_train_dataloader), enumerate(text_train_dataloader)):
    optimizer.zero_grad()
    X, token_ids, valid_length, segment_ids, Y = X.to(device), token_ids.to(device), valid_length.to(device), segment_ids.to(device), Y.to(device)
    outputs = model3(X,token_ids, valid_length, segment_ids)
    loss = criterion(outputs, Y)
    loss.backward()

    optimizer.step()

    cost += loss
    if i % 10 == 9:
      print(f'epoch: {epoch} | batch_id: {i} | loss: {loss.item()}') # 밑에 test로 수정
  for (i, (X, Y)), (_, (token_ids, valid_length, segment_ids, _)) in zip(enumerate(audio_test_dataloader), enumerate(text_test_dataloader)):
    X, token_ids, valid_length, segment_ids, Y = X.to(device), token_ids.to(device), valid_length.to(device), segment_ids.to(device), Y.to(device)
    outputs = model3(X,token_ids, valid_length, segment_ids)
    total += Y.size(0)
    correct += sum(torch.argmax(outputs, 1) == Y)
  avg_cost = cost / len(audio_train_dataloader)
  accuracy = 100*correct/total

  print(f'\nepoch: {epoch} | loss: {avg_cost}\naccuracy: {accuracy}\n')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


KeyboardInterrupt: ignored

In [ ]:
# # Empty CUDA
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# torch.save(model3,'/content/drive/MyDrive/Colab Notebooks/ai/multimodal_emotion_classification.pt')

In [ ]:
y_pred = np.array([])
y_test = np.array([])
for (i, (X, Y)), (_, (token_ids, valid_length, segment_ids, _)) in zip(enumerate(audio_test_dataloader), enumerate(text_test_dataloader)):
  X, token_ids, valid_length, segment_ids, Y = X.to(device), token_ids.to(device), valid_length.to(device), segment_ids.to(device), Y.to(device)
  outputs = model3(X,token_ids, valid_length, segment_ids)
  y_pred = np.append(y_pred, np.array(torch.argmax(outputs,1).cpu()))
  y_test = np.append(y_test, np.array(Y.cpu()))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
print(classification_report(y_test, y_pred))  # class 한글로 뜨게

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.89       612
         1.0       0.89      0.85      0.87       340
         2.0       0.85      0.83      0.84       287
         3.0       0.85      0.83      0.84       359
         4.0       0.75      0.78      0.76       223

    accuracy                           0.85      1821
   macro avg       0.84      0.84      0.84      1821
weighted avg       0.85      0.85      0.85      1821



실제 데이터 모델 성능 테스트

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/ai/완성

/content/drive/MyDrive/Colab Notebooks/ai/완성


In [ ]:
#감성대화정보 엑셀로 불러오기
data_pre= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/감성대화/감성대화말뭉치_table.xlsx')

# 감정데이터중에 필요없는 카테고리('당황') 제거해주기
data_sub = data_pre.loc[(data_pre['감정_대분류'] == "기쁨") | (data_pre['감정_소분류'] == "안달하는") | (data_pre['감정_소분류'] == "성가신") | (data_pre['감정_소분류'] == "툴툴대는") | (data_pre['감정_소분류'] == "방어적인") | (data_pre['감정_소분류'] == "혐오스러운")
 | (data_pre['감정_소분류'] == "비통한") | (data_pre['감정_소분류'] == "슬픔") | (data_pre['감정_소분류'] == "우울한") | (data_pre['감정_소분류'] == "실망한") | (data_pre['감정_소분류'] == "후회되는") | (data_pre['감정_소분류'] == "눈물이 나는")
 | (data_pre['감정_소분류'] == "걱정스러운") | (data_pre['감정_소분류'] == "조심스러운") | (data_pre['감정_소분류'] == "초조한") | (data_pre['감정_소분류'] == "두려운") | (data_pre['감정_소분류'] == "스트레스 받는")
 | (data_pre['감정_소분류'] == "고립된") | (data_pre['감정_소분류'] == "염세적인") | (data_pre['감정_소분류'] == "상처") 
 ]

print(data_sub['감정_대분류'].value_counts())

기쁨    827
슬픔    590
불안    379
분노    339
상처    167
당황    126
Name: 감정_대분류, dtype: int64


In [ ]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
data_sub = data_sub.copy()
data_sub.loc[(data_sub['감정_소분류'] == "안달하는") | (data_sub['감정_소분류'] == "성가신") | (data_sub['감정_소분류'] == "툴툴대는") | (data_sub['감정_소분류'] == "방어적인") | (data_sub['감정_소분류'] == "혐오스러운"), 'Emotion'] = '분노'  #분노
data_sub.loc[(data_sub['감정_소분류'] == "비통한") | (data_sub['감정_소분류'] == "슬픔") | (data_sub['감정_소분류'] == "우울한") | (data_sub['감정_소분류'] == "실망한") | (data_sub['감정_소분류'] == "후회되는") | (data_sub['감정_소분류'] == "눈물이 나는"), 'Emotion'] = '우울'  #우울 
data_sub.loc[(data_sub['감정_소분류'] == "걱정스러운") | (data_sub['감정_소분류'] == "조심스러운") | (data_sub['감정_소분류'] == "초조한") | (data_sub['감정_소분류'] == "두려운") | (data_sub['감정_소분류'] == "스트레스 받는"), 'Emotion'] = '불안'  #불안 
data_sub.loc[(data_sub['감정_소분류'] == "고립된") | (data_sub['감정_소분류'] == "염세적인") | (data_sub['감정_소분류'] == "상처")  , 'Emotion'] = '자살'  #자살 
data_sub.loc[(data_sub['감정_대분류'] == "기쁨"), 'Emotion'] = '기쁨' #기쁨

In [ ]:
# 음성데이터 라벨 및 경로데이터만 뽑아오기
data_path = data_sub.loc[:,['Emotion','NO.']]
data_path.columns = ['Emotions','Path']

# 음성데이터 경로수정해주기(압축푼 폴더로 경로수정)
data_path['Path'] = '/content/drive/MyDrive/Colab Notebooks/감성대화/원천데이터/감성대화말뭉치AI데이터_Wave_남자성우_5000/'+data_path['Path']+'.wav'

data_path.head(10)

,Emotions,Path
84,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
85,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
86,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
87,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
88,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
89,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
90,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
91,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
92,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
93,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...


In [ ]:
data_path = data_path.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset
data_path

,Emotions,Path
0,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
1,분노,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
2,자살,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
3,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
4,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
...,...,...
2423,불안,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
2424,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
2425,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...
2426,기쁨,/content/drive/MyDrive/Colab Notebooks/감성ᄃ...


In [ ]:
target = data_path['Path'].iloc[1]
target


'/content/drive/MyDrive/Colab Notebooks/감성대화/원천데이터/감성대화말뭉치AI데이터_Wave_남자성우_5000/M_002240.wav'

In [ ]:
r = sr.Recognizer()
harvard = sr.AudioFile(target)
with harvard as source:
  audio = r.record(source)
text = r.recognize_google(audio, language='ko-KR')

In [ ]:
text

'자녀들이 손자를 봐 달라고 자꾸 조르네'

In [ ]:
# 음성의 특성추출하는 함수 ( MFCC, MEL, RMSV)
def extract_features(data):

    result = np.array([])

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
#    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
#    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return result

In [ ]:
data, sample_rate = librosa.load(target, duration=2.5, offset=0.6)

In [ ]:
feature = np.array(extract_features(data))

In [ ]:
x = feature.reshape(1,-1)
z = text

In [ ]:
# # 음성 데이터 스케일 조정
with open('/content/drive/MyDrive/Colab Notebooks/ai/scaler.pkl','rb') as f:
  scaler = pickle.load(f)
x = scaler.transform(x)

In [ ]:
# 음성 데이터의 차원 모델에 맞게 통일 
x = np.expand_dims(x, axis=2)
x.shape

(1, 148, 1)

In [ ]:
X = torch.tensor(x, dtype=torch.float32)

#문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
with open('/content/drive/MyDrive/Colab Notebooks/ai/kobert_data.pkl', 'rb') as f:
  bertmodel, vocab, _ = pickle.load(f)
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

[██████████████████████████████████████████████████]


In [ ]:

transform = nlp.data.BERTSentenceTransform(
            tok, max_seq_length=50, pad=True, pair=False)

Z = list(transform(z))
Z1, Z2, Z3 = torch.tensor(Z[0].reshape(1,-1)),torch.tensor(Z[1].reshape(1,-1)),torch.tensor(Z[2].reshape(1,-1))


X.shape, Z1.shape,Z2.shape,Z3.shape

(torch.Size([1, 148, 1]),
 torch.Size([1, 50]),
 torch.Size([1, 1]),
 torch.Size([1, 50]))

In [ ]:
# 음성 모델 만들기

class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=x_train.shape[1], out_channels=256, kernel_size=1, stride=1,padding='same')
        self.max_pool1 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding='same')
        self.max_pool2 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv3 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=5, stride=1, padding='same')
        self.max_pool3 = nn.MaxPool1d(5, stride=2, padding = 2)
        self.dropout1 = nn.Dropout(0.2)
        
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding='same')
        self.max_pool4 = nn.MaxPool1d(5, stride=2, padding = 2)
        
        self.fc1 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)

        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)

        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        x = self.dropout1(x)

        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)

        x = torch.flatten(x,1)

        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        
        return x

In [ ]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=32,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 멀티모달 모델 만들기

class MultimodalClassifier(nn.Module):
    def __init__(self, audioModel, textModel):
        super(MultimodalClassifier, self).__init__()
        self.audioModel = audioModel
        self.textModel = textModel

        self.fc1 = nn.Linear(64,32)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(32, 5)

    def forward(self, x1, token_ids, valid_length, segment_ids):
        x1 = self.audioModel(x1)
        x2 = self.textModel(token_ids, valid_length, segment_ids)
        x = torch.cat((x1,x2), 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        
        x = F.relu(self.fc2(x))
        return x

In [ ]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/ai/multimodal_emotion_classification.pt')

In [ ]:
output = model(X.to(device), Z1.to(device), Z2.to(device),Z3.to(device))

In [ ]:
output

In [ ]:
le.inverse_transform([torch.argmax(output.cpu())])

array(['우울'], dtype=object)